In [15]:
import requests
from bs4 import BeautifulSoup as soup
import time
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import os

---

### Get one song: George Ezra - Budapest

In [ ]:
html = requests.get('https://www.lyrics.com/lyric/18961542/Queen/The+Show+Must+Go+On+%5B%2A%5D')

In [ ]:
page = soup(html.text, 'html.parser')

In [ ]:
refined_text = page.find_all(attrs={'class':'lyric-body'})

In [ ]:
refined_text[0].text

---

### Get all songs and lyrics from Queen

In [2]:
## Download the website that contains all songs of the artist
html = requests.get('https://www.lyrics.com/artist/Queen/5205')

In [3]:
## Parse the HTML site with BeautifulSoup
urls_page = soup(html.text, 'html.parser')

In [4]:
## Extract all tags with 'a' from the parsed through html file
a = urls_page.find_all('a')

In [ ]:
## Get the attribute out of an href for one tag 'a' (example) - this gives us the URL for a specific song
a[70].get('href')

In [5]:
## Get all the URLs for Queen songs on Lyrics.com
urls = []
for i in a:
    if '/lyric/' in i.get('href'):
        urls.append(i.get('href'))

In [6]:
## Get all the titles for Queen songs on Lyrics.com
titles = []
for i in a:
    if '/lyric/' in i.get('href'):
        titles.append(i.get_text())

In [8]:
## Create empty list for lyrics to be filled in
lyrics = []

## Put the lyrics in a list
for i in tqdm(urls):
    url = 'https://www.lyrics.com' + i
    html = requests.get(url)
    page = soup(html.text, 'html.parser')
    try:
        refined_text = page.find_all(attrs={'class':'lyric-body'})
        lyrics.append(refined_text[0].text)
    except:
        lyrics.append(np.NaN)
    time.sleep(1)

100%|██████████| 26/26 [01:29<00:00,  3.34s/it]


In [ ]:
## Put all three lists into a dataframe
df = pd.DataFrame({'URL':['https://www.lyrics.com' + i for i in urls],'Title':titles, 'Lyrics': lyrics})

In [ ]:
## Write dataframe to a csv file
df.to_csv('Queen_lyrics.csv')

---

### Create a function that takes a URL of an artist discography and outputs a dataframe with lyrics

In [ ]:
def get_lyrics(url):
    """
    This function takes a URL from a selected artist on Lyrics.com and outputs a dataframe including all its songs lyrics
    """
    ## Download website under url
    html = requests.get(url)
    
    ## Parse the HTML site with BeautifulSoup
    urls_page = soup(html.text, 'html.parser')
    
    ## Extract all tags with 'a' from the parsed through html file
    a = urls_page.find_all('a')
    
    ## Get all the URLs and titles for the artists' songs on Lyrics.com
    urls = []
    titles = []
    for i in a:
        if '/lyric/' in i.get('href'):
            urls.append(i.get('href'))
            titles.append(i.get_text())
            
    ## Download all the lyrics for all the songs listed from the artist
    lyrics = []
    
    for i in urls:
        url = 'https://www.lyrics.com' + i
        try:
            html = requests.get(url)
            page = soup(html.text, 'html.parser')
            try:
                refined_text = page.find_all(attrs={'class':'lyric-body'})
                lyrics.append(refined_text[0].text)
            except:
                lyrics.append(np.NaN)
        except:
            pass
        time.sleep(1)
        
    ## Create a dataframe out of the three lists
    
    df = pd.DataFrame({'URL':['https://www.lyrics.com' + i for i in urls],'Title':titles, 'Lyrics': lyrics})
    
    return df

In [ ]:
## Get lyrics for The Rolling Stones
df_rolling = get_lyrics('https://www.lyrics.com/artist/The+Rolling+Stones/5298')

In [ ]:
## Write dataframe to a csv file
df_rolling.to_csv('RollingStones_lyrics.csv')

In [ ]:
## Get lyrics for The Rolling Stones
df_miley = get_lyrics('https://www.lyrics.com/artist/Miley+Cyrus/823418')

In [ ]:
## Write dataframe to a csv file
df_miley.to_csv('Output/MileyCyrus_lyrics.csv')

---

### Write a small program to search for any name and get back the lyrics of that artist from Lyrics.com

This will only work with the (maximum) 6 top options that Lyrics returns

In [ ]:
### Load necessary packages for the program to run
import requests
from bs4 import BeautifulSoup as soup
import time
import pandas as pd
import numpy as np
import re

In [ ]:
### Initiate user search and prompt user for input
user_search = input('What do you want to search for?')

In [ ]:
### Strip user input and replace user input whitespaces with '%20' that is used as space on Lyrics.com
user_search = user_search.strip()
user_search = re.sub('( ){1,}', '%20', user_search)
user_search = user_search.lower()

### Create link to be used for going to search results site on Lyrics.com
searchlink = 'https://www.lyrics.com/lyrics/' + user_search

In [ ]:
### Parse the website with beautifulsoup
html = requests.get(searchlink)
searchpage = soup(html.text, 'html.parser')

a = searchpage.find_all('a')

artist_urls = []
artist_names = []

for i in a:
    if 'artist/' in i.get('href'):
        artist_urls.append('https://www.lyrics.com/' + i.get('href'))
        artist_names.append(i.get('title'))

In [ ]:
## Create options dataframe (there are duplicates and we want to select only the first nine occurances)
artist_opts = pd.DataFrame({'URL':artist_urls[0:18:2],'Name':artist_names[0:18:2]})

In [ ]:
## Print out each element in my artist_names list for the user to choose from
print('The following artists were found on Lyrics.com for your search:')
for i, name in enumerate(artist_opts['Name']):
    print(str(i+1) + ') ' + str(name))

In [ ]:
## Prompt user for input and confirm the input of the user is among the listed options
while True:
    try:
        user_selection = input('Please type the number of the artist you want to load lyrics from: ')
        if int(user_selection) in list((artist_opts.index.values + 1)):
            break
        else:
            raise ValueError
    except ValueError:
        print("Not a valid option! Please try again ...")

## Print an empty line and then the user selected artist name
print()
print(f'Great, I will search lyrics for: {artist_opts.iloc[int(user_selection)-1, 1]}')

In [ ]:
### Save selected artist url in artist_url variable
artist_url = artist_opts.iloc[int(user_selection)-1, 0]

In [ ]:
### Run the user defined function that scrapes the website for lyrics and puts it into a dataframe
lyrics = get_lyrics(artist_url)

In [ ]:
string = artist_opts.iloc[int(user_selection)-1, 1]
artist_shortened = ''.join(e for e in string if e.isalnum())

In [ ]:
### Write out dataframe to a csv output file
filename = f'{artist_shortened}_lyrics.csv'
lyrics.to_csv(filename)

---

### Combine all this into one lyrics downloader program
--> This program is saved as LyricsSearch.py to be run directly from the Terminal (try running it with your favorite singer!)

In [1]:
### Load necessary packages for the program to run
import requests
from bs4 import BeautifulSoup as soup
import time
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm

### Create user defined function for scraping lyrics
def get_lyrics(urls):
    """
    This function takes a urls from an artist on Lyrics.com and outputs a dataframe including all its songs lyrics
    """
 
    ## Download all the lyrics for all the songs listed from the artist
    lyrics = []

    ## For loop including a tqdm progress bar (super cool!)
    for i in tqdm(urls):
        url = 'https://www.lyrics.com' + i
        try:
            html = requests.get(url)
            page = soup(html.text, 'html.parser')
            try:
                refined_text = page.find_all(attrs={'class':'lyric-body'})
                lyrics.append(refined_text[0].text)
            except:
                lyrics.append(np.NaN)
        except:
            pass
        time.sleep(1)

    ## Create a dataframe out of the three lists

    df = pd.DataFrame({'URL':['https://www.lyrics.com' + i for i in urls],'Title':titles, 'Lyrics': lyrics})

    return df


### Initiate user search and prompt user for input
user_search = input('What do you want to search for?')

### Strip user input and replace user input whitespaces with '%20' that is used as space on Lyrics.com
user_search = user_search.strip()
user_search = re.sub('( ){1,}', '%20', user_search)
user_search = user_search.lower()

### Create link to be used for going to search results site on Lyrics.com
searchlink = 'https://www.lyrics.com/lyrics/' + user_search

### Parse the website with beautifulsoup
html = requests.get(searchlink)
searchpage = soup(html.text, 'html.parser')

a = searchpage.find_all('a')

artist_urls = []
artist_names = []

for i in a:
    if 'artist/' in i.get('href'):
        artist_urls.append('https://www.lyrics.com/' + i.get('href'))
        artist_names.append(i.get('title'))

## Create options dataframe (there are duplicates and we want to select only the first nine occurances)
artist_opts = pd.DataFrame({'URL':artist_urls[0:18:2],'Name':artist_names[0:18:2]})

## Print out each element in my artist_names list for the user to choose from
print()
print('The following artists were found on Lyrics.com for your search:')
for i, name in enumerate(artist_opts['Name']):
    print(str(i+1) + ') ' + str(name))

print()

## Prompt user for input and confirm the input of the user is among the listed options
while True:
    try:
        user_selection = input('Please type the number of the artist you want to load lyrics from: ')
        if int(user_selection) in list((artist_opts.index.values + 1)):
            break
        else:
            raise ValueError
    except ValueError:
        print("Not a valid option! Please try again ...")
        
### Print an empty line and then the user selected artist name
print()
print(f'Great, I will search lyrics for {artist_opts.iloc[int(user_selection)-1, 1]}')

### Save selected artist url in artist_url variable
artist_url = artist_opts.iloc[int(user_selection)-1, 0]

## Download artist website under url
html = requests.get(artist_url)

## Parse the HTML site with BeautifulSoup
urls_page = soup(html.text, 'html.parser')

## Extract all tags with 'a' from the parsed through html file
a = urls_page.find_all('a')

## Get all the URLs and titles for the artists' songs on Lyrics.com
urls = []
titles = []
for i in a:
    if '/lyric/' in i.get('href'):
        urls.append(i.get('href'))
        titles.append(i.get_text())

### Estimate time it will take to load all lyrics and confirm with user
est = len(urls) * 3
est_min = est // 60
est_sec = est % 60
print()
print(f'It will take approximately {est_min} minutes {est_sec} seconds to load all {len(urls)} songs from this artist.')
print()

while True:
    try:
        confirm = input('Do you want to continue? (Y/N)')
        if confirm in ['Y','N']:
            break
        else:
            raise ValueError
    except ValueError:
        print("Not a valid option! Please try again with Y or N...")

if confirm == 'N':
    print('Okay then, quitting the program...')
    quit()

else:  
    ### Run the user defined function that scrapes the website for lyrics and puts it into a dataframe
    ### A tqdm progress bar is included in the defined function above
    lyrics = get_lyrics(urls)

    ### Print message
    print()
    print('All lyrics loaded into dataframe.')

    ### Strip down the artist name to be used for filename
    string = artist_opts.iloc[int(user_selection)-1, 1]
    artist_shortened = ''.join(e for e in string if e.isalnum())

    ### Write out dataframe to a csv output file
    filename = f'{artist_shortened}_lyrics.csv'
    lyrics.to_csv(filename)

    dirpath = os.getcwd()
    filepath = dirpath + '/' + filename

    ### Print message
    print()
    print(f'CSV file created with all lyrics for artist {string}.')
    print(f'The file is saved in the following location:  {filepath}')
    print()

What do you want to search for? Gotye


The following artists were found on Lyrics.com for your search:
1) Gotye


Please type the number of the artist you want to load lyrics from:  1



Great, I will search lyrics for: Gotye

It will take approximately 1 minutes 18 seconds to load all 26 songs from this artist.



Do you want to continue? (Y/N) Y


100%|██████████| 26/26 [01:30<00:00,  3.42s/it]



All lyrics loaded into dataframe.


NameError: name 'os' is not defined